In [ ]:
pip install -U sagemaker

In [ ]:
pip install seaborn

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [5]:
dataset = pd.read_csv('HotelReservationsModified.csv')
dataset

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,...,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled,label_avg_price_per_room
0,INN00001,2,0,1,2,0,224,2017,10,2,...,False,False,False,False,False,True,False,False,True,1
1,INN00002,2,0,2,3,0,5,2018,11,6,...,False,False,False,False,False,False,True,False,True,2
2,INN00003,1,0,2,1,0,1,2018,2,28,...,False,False,False,False,False,False,True,True,False,1
3,INN00004,2,0,0,2,0,211,2018,5,20,...,False,False,False,False,False,False,True,True,False,2
4,INN00005,2,0,1,1,0,48,2018,4,11,...,False,False,False,False,False,False,True,True,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,0,85,2018,8,3,...,False,False,False,False,False,False,True,False,True,3
36271,INN36272,2,0,1,3,0,228,2018,10,17,...,False,False,False,False,False,False,True,True,False,2
36272,INN36273,2,0,2,6,0,148,2018,7,1,...,False,False,False,False,False,False,True,False,True,2
36273,INN36274,2,0,0,3,0,63,2018,4,21,...,False,False,False,False,False,False,True,True,False,2


In [6]:
dataset = dataset.drop(['Booking_ID'], axis=1)

label_column = dataset['label_avg_price_per_room']

def transformLabel(label):
    return label - 1

new_label_column = label_column.apply(transformLabel)
dataset['label_avg_price_per_room'] = new_label_column

dataset

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Aviation,market_segment_type_Complementary,market_segment_type_Corporate,market_segment_type_Offline,market_segment_type_Online,booking_status_Canceled,booking_status_Not_Canceled,label_avg_price_per_room
0,2,0,1,2,0,224,2017,10,2,0,...,False,False,False,False,False,True,False,False,True,0
1,2,0,2,3,0,5,2018,11,6,0,...,False,False,False,False,False,False,True,False,True,1
2,1,0,2,1,0,1,2018,2,28,0,...,False,False,False,False,False,False,True,True,False,0
3,2,0,0,2,0,211,2018,5,20,0,...,False,False,False,False,False,False,True,True,False,1
4,2,0,1,1,0,48,2018,4,11,0,...,False,False,False,False,False,False,True,True,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,0,85,2018,8,3,0,...,False,False,False,False,False,False,True,False,True,2
36271,2,0,1,3,0,228,2018,10,17,0,...,False,False,False,False,False,False,True,True,False,1
36272,2,0,2,6,0,148,2018,7,1,0,...,False,False,False,False,False,False,True,False,True,1
36273,2,0,0,3,0,63,2018,4,21,0,...,False,False,False,False,False,False,True,True,False,1


In [7]:
X = dataset.iloc[:, :-1].values
X = np.array(X).astype('float32')
X

array([[2., 0., 1., ..., 0., 0., 1.],
       [2., 0., 2., ..., 1., 0., 1.],
       [1., 0., 2., ..., 1., 1., 0.],
       ...,
       [2., 0., 2., ..., 1., 0., 1.],
       [2., 0., 0., ..., 1., 1., 0.],
       [2., 0., 1., ..., 0., 0., 1.]], dtype=float32)

In [8]:
X.shape

(36275, 31)

In [9]:
Y = dataset.iloc[:, -1].values
Y = np.array(Y).astype('float32')
Y

array([0., 1., 0., ..., 1., 1., 2.], dtype=float32)

In [10]:
Y.shape

(36275,)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [12]:
X_train.shape, X_test.shape

((25392, 31), (10883, 31))

In [13]:
Y_train.shape, Y_test.shape

((25392,), (10883,))

In [22]:
import sagemaker
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.amazon.amazon_estimator import get_image_uri
import boto3
from sagemaker import Session
import sagemaker.amazon.common as smac
import io
import os

In [ ]:
session = sagemaker.Session()
bucket = 'sprint0405'
subpasta_modelo = 'modelos/hotelreservations/linear-learner'
subpasta_dataset = 'datasets/hotelreservations'
key = 'hotelreservations-train-data'
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)

In [24]:
buffer = io.BytesIO()
smac.write_numpy_to_dense_tensor(buffer, X_train, Y_train)
buffer.seek(0)

0

In [25]:
boto3.resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key)).upload_fileobj(buffer)

In [29]:
knn = sagemaker.estimator.Estimator(
        sagemaker.image_uris.retrieve(framework='knn', region = 'us-east-1'),
        role,
        instance_count=1,
        instance_type="ml.m4.xlarge",
        output_path=output_location,
        sagemaker_session=sagemaker.Session(),
    )

In [33]:
hyperparams = {"feature_dim": 31, "k": 3, "sample_size": X_test.shape[0], "predictor_type": "classifier"}
knn.set_hyperparameters(**hyperparams)

In [ ]:
knn.fit({"train": s3_train_data})

In [ ]:
knn_classifier = knn.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

In [36]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import json

knn_classifier.serializer = CSVSerializer()
knn_classifier.deserializers = JSONDeserializer()

In [37]:
X_test.shape

(10883, 31)

In [ ]:
results = json.loads(knn_classifier.predict(X_test))
results['predictions']

In [39]:
predictions = np.array([result['predicted_label'] for result in results['predictions']])
predictions

array([2., 0., 1., ..., 0., 2., 2.])

In [40]:
Y_test = np.array(Y_test).astype(int)
Y_test

array([2, 0, 1, ..., 2, 2, 2])

In [41]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [42]:
accuracy_score(Y_test, predictions)

0.6182118901038317

In [43]:
cm = confusion_matrix(Y_test, predictions)
cm

array([[2122,  781,  355],
       [ 874, 2426,  706],
       [ 503,  936, 2180]])

In [44]:
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.61      0.65      0.63      3258
           1       0.59      0.61      0.60      4006
           2       0.67      0.60      0.64      3619

    accuracy                           0.62     10883
   macro avg       0.62      0.62      0.62     10883
weighted avg       0.62      0.62      0.62     10883

